In [4]:
import keras

Using TensorFlow backend.


In [5]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [16]:
import zipfile
with zipfile.ZipFile("./sample_data/train.zip", 'r') as zip_ref:
    zip_ref.extractall("./sample_data/train")

with zipfile.ZipFile("./sample_data/test.zip", 'r') as zip_ref:
    zip_ref.extractall("./sample_data/test")


In [3]:
import numpy as np
from keras.preprocessing.image import load_img
image=load_img("./sample_data/Data/Data/angry/PrivateTest_94431780.jpg")
image.size

(48, 48)

In [28]:

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense,BatchNormalization

model = Sequential()
model.add(Conv2D(64,  (3, 3), padding="same", input_shape=(48,48,1)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(128,  (3, 3), padding="same",))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(256,  (3, 3), padding="same",))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(512,  (3, 3), padding="same",))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))

model.add(Conv2D(1024,  (3, 3), padding="same",))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))


model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors

model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.2))

model.add(Dense(7,activation="softmax"))

model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_43 (Conv2D)           (None, 48, 48, 64)        640       
_________________________________________________________________
batch_normalization_43 (Batc (None, 48, 48, 64)        256       
_________________________________________________________________
activation_59 (Activation)   (None, 48, 48, 64)        0         
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
dropout_15 (Dropout)         (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_44 (Conv2D)           (None, 24, 24, 128)       73856     
_________________________________________________________________
batch_normalization_44 (Batc (None, 24, 24, 128)      

In [8]:
pip install livelossplot

In [30]:
from keras.preprocessing.image import ImageDataGenerator
#from tensorflow.keras.callbacks import ModelCheckpoint  gives Error :'Sequential' object has no attribute '_in_multi_worker_mode'
#instead use
from keras.callbacks import ModelCheckpoint

batch_size=64
image_size=48
epochs=30

train_datagen = ImageDataGenerator(
        rescale=1./255,
#        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
#        vertical_flip=True)
#        width_shift_range=0.1,
#        height_shift_range=0.1)

validation_datagen = ImageDataGenerator(
        rescale=1./255,
#        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
#        vertical_flip=True)
#        width_shift_range=0.1,
#        height_shift_range=0.1)

train_generator = train_datagen.flow_from_directory(
        'sample_data/train/train',  
        color_mode="grayscale",
        target_size=(image_size,image_size),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True)  

validation_generator = validation_datagen.flow_from_directory(
        "sample_data/test/test",
        target_size=(image_size,image_size),
        color_mode="grayscale",
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

checkpoint = ModelCheckpoint("./sample_data/model_weights.h5", monitor='val_accuracy',
                             save_weights_only=True, mode='max', verbose=1)
callbacks=[checkpoint]

model.fit_generator(
        generator=train_generator,
        validation_data=validation_generator,
        epochs=epochs,
        callbacks=callbacks)




#model.save_weights('./sample_data/first_try.h5')  # always save your weights after training or during training

Found 28708 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.
Epoch 1/30
449/449 [==============================] - 40s 89ms/step - loss: 1.7927 - accuracy: 0.2649 - val_loss: 2.8008 - val_accuracy: 0.2797

Epoch 00001: saving model to ./sample_data/model_weights.h5
Epoch 2/30
449/449 [==============================] - 39s 86ms/step - loss: 1.5632 - accuracy: 0.3850 - val_loss: 1.2763 - val_accuracy: 0.3670

Epoch 00002: saving model to ./sample_data/model_weights.h5
Epoch 3/30
449/449 [==============================] - 38s 86ms/step - loss: 1.4285 - accuracy: 0.4462 - val_loss: 1.3188 - val_accuracy: 0.4730

Epoch 00003: saving model to ./sample_data/model_weights.h5
Epoch 4/30
449/449 [==============================] - 38s 85ms/step - loss: 1.3512 - accuracy: 0.4830 - val_loss: 1.5554 - val_accuracy: 0.4264

Epoch 00004: saving model to ./sample_data/model_weights.h5
Epoch 5/30
449/449 [==============================] - 38s 85ms/step - loss: 1.3065 - accuracy: 

In [33]:
#Saving model architecture as json
model_json = model.to_json()
with open("./sample_data/model.json", "w") as json_file:
    json_file.write(model_json)